In [176]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SolveLinSys
from numba import njit

PDE:

\begin{align*}
0 = \max_{\tilde{e}} & b \delta \eta \log \tilde{e} + b(\eta-1)\left[\gamma_1 + \gamma_2 y + \gamma_2^+\boldsymbol{1}_{y\geq \bar{y}}(y-\bar{y})\right] \tilde{e}  \theta\\
& + \frac{\partial V}{\partial y}\tilde{e} \theta - b \delta \frac{\partial V}{\partial b}\\
& + \frac{1}{2}\frac{\partial^2 V}{(\partial y)^2}|\sigma_y|^2(\tilde{e})^2 - \ell \tilde{e}\\
& + b(\eta-1)\left[\frac{1}{2}\left(\gamma_2 + \gamma_2^+\boldsymbol{1}_{y\geq \bar{y}}\right)\right]|\sigma_y|^2(\tilde{e})^2\\
\end{align*}

The FOC of $\tilde{e}$ gives:

$$
\tilde{e}^* = \frac{-\left[\theta G(y) - \ell\right] - \sqrt{\left[ \theta G(y) - \ell\right]^2-4b\delta \eta |\sigma_y|^2F(y)}}{2|\sigma_y|^2F(y)}
$$

where $G(y):=\frac{\partial V}{\partial y} + b(\eta-1)\left[\gamma_1 + \gamma_2 y + \gamma_2^+\boldsymbol{1}_{y\geq \bar{y}}(y-\bar{y})\right]$, $F(y):=\frac{\partial^2 V}{(\partial y)^2} + b(\eta-1)\left(\gamma_2 + \gamma_2^+\boldsymbol{1}_{y\geq \bar{y}}\right)$.


In [177]:
@njit
def compute_derivatives(data, dim, order, dlt):
    res = np.zeros_like(data)
    if order == 1:                    # first order derivatives
        if dim == 0:                  # to first dimension
            res[1:-1,:] = (1 / (2 * dlt)) * (data[2:,:] - data[:-2,:])
            res[-1,:] = (1 / dlt) * (data[-1,:] - data[-2,:])
            res[0,:] = (1 / dlt) * (data[1,:] - data[0,:])
        elif dim == 1:                # to second dimension
            res[:,1:-1] = (1 / (2 * dlt)) * (data[:,2:] - data[:,:-2])
            res[:,-1] = (1 / dlt) * (data[:,-1] - data[:,-2])
            res[:,0] = (1 / dlt) * (data[:,1] - data[:,0])        
    elif order == 2:
        if dim == 0:                  # to first dimension
            res[1:-1,:] = (1 / dlt ** 2) * (data[2:,:] + data[:-2,:] - 2 * data[1:-1,:])
            res[-1,:] = (1 / dlt ** 2) * (data[-1,:] + data[-3,:] - 2 * data[-2,:])
            res[0,:] = (1 / dlt ** 2) * (data[2,:] + data[0,:] - 2 * data[1,:])
        elif dim == 1:                # to second dimension
            res[:,1:-1] = (1 / dlt ** 2) * (data[:,2:] + data[:,:-2] - 2 * data[:,1:-1])
            res[:,-1] = (1 / dlt ** 2) * (data[:,-1] + data[:,-3] - 2 * data[:,-2])
            res[:,0] = (1 / dlt ** 2) * (data[:,2] + data[:,0] - 2 * data[:,1])    
    return res

def false_transient_one_iteration(stateSpace, A, B_y, B_b, C_yy, C_bb, D, v0, ε=.3, tol=-10):
    A = A.reshape(-1, 1, order='F')
    B = np.hstack([B_y.reshape(-1, 1, order='F'), B_b.reshape(-1, 1, order='F')])
    C = np.hstack([C_yy.reshape(-1, 1, order='F'), C_bb.reshape(-1, 1, order='F')])
    D = D.reshape(-1, 1, order='F')
    v0 = v0.reshape(-1, 1, order='F')
    out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε, tol)
    return out

In [200]:
def false_transient(y_grid, b_grid, ell, model_paras=(), v0=None, ϵ=.5, tol=1e-8, max_iter=10_000, print_all=True):
    η, δ, θ, πc_o, σ_y, γ_1, γ_2, γ_2p, y_bar = model_paras
    Δ_y = y_grid[1] - y_grid[0]
    Δ_b = b_grid[1] - b_grid[0]
    (y_mat, b_mat) = np.meshgrid(y_grid, b_grid, indexing = 'ij')
    stateSpace = np.hstack([y_mat.reshape(-1, 1, order='F'), b_mat.reshape(-1, 1, order='F')])    

    if v0 is None:
        v0 = -δ*η*(y_mat+y_mat**2)

    d_Λ = γ_1 + γ_2*y_mat + γ_2p*(y_mat>y_bar)*(y_mat-y_bar)
    dd_Λ = γ_2 + γ_2p*(y_mat>y_bar)

    count = 0
    error = 1.

    while error > tol and count < max_iter:
        v_old = v0.copy()

        v0_dy = compute_derivatives(v0, 0, 1, Δ_y)
        v0_dyy = compute_derivatives(v0, 0, 2, Δ_y)
        v0_db = compute_derivatives(v0, 1, 1, Δ_b)

        G = v0_dy + b_mat*(η-1)*d_Λ

        if σ_y == 0:
            e_tilde = -b_mat*δ*η/(G*θ-ell)
        else:
            temp = σ_y**2*(v0_dyy+b_mat*(η-1.)*dd_Λ)
            root = (θ*G - ell)**2 - 4*b_mat*δ*η*temp
            root[root<0] = 0.
            e_tilde = (-(G*θ-ell) - np.sqrt(root)) / (2*temp)
        e_tilde[e_tilde<=0] = 1e-16

        A = np.zeros_like(y_mat)
        B_y = e_tilde * θ
        B_b = -b_mat*δ
        C_yy = .5 * σ_y**2 * e_tilde**2
        C_bb = np.zeros_like(y_mat)
        D = b_mat*δ*η*np.log(e_tilde) + b_mat*(η-1)*d_Λ*e_tilde*θ\
            + .5*b_mat*(η-1)*dd_Λ*σ_y**2*e_tilde**2 - ell*e_tilde
        res = false_transient_one_iteration(stateSpace, A, B_y, B_b, C_yy, C_bb, D, v0, ϵ) # CG tol = 1e-10
        v0 = res[2].reshape(v0.shape, order = "F")

        rhs_error = A*v0 + B_y*v0_dy + B_b*v0_db + C_yy*v0_dyy + D
        rhs_error = np.max(abs(rhs_error))
        lhs_error = np.max(abs((v0 - v_old)/ϵ))
        error = lhs_error
        count += 1
        if print_all:
            print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))

    print("Converged. Total iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))

    res = {'v0': v0,
           'v0_dy': v0_dy,
           'v0_dyy': v0_dyy,
           'e_tilde': e_tilde,
           'y_grid': y_grid,}
    return res

In [210]:
η = .032
δ = .01

θ = pd.read_csv('../data/model144.csv', header=None).to_numpy()[:, 0]/1000.
πc_o = np.ones_like(θ)/len(θ)
σ_y = 1.2*np.mean(θ)

y_bar = 2.
γ_1 = 1.7675/10000
γ_2 = .0022*2
# γ_2p = .0197*2*.5 # weighted model of low and high
γ_2p = 0. # low damage model

n_y = 100
y_min = 0.
y_max = 4.
y_grid = np.linspace(y_min, y_max, n_y)

n_b = 100
b_min = 1./n_b
b_max = 1.
b_grid = np.linspace(b_min, b_max, n_b)

model_paras = (η, δ, np.mean(θ), πc_o, σ_y, γ_1, γ_2, γ_2p, y_bar) 

ell = 1e-3
ell_step = ell/100.

model_res = false_transient(y_grid, b_grid, ell, model_paras=model_paras, v0=None, ϵ=1.,
                            tol=1e-6, max_iter=1_000, print_all=False)
model_res_right = false_transient(y_grid, b_grid, ell+ell_step, model_paras=model_paras, v0=None, ϵ=1.,
                            tol=1e-6, max_iter=1_000, print_all=False)

Converged. Total iteration 1000: LHS Error: 4.517907913934027e-06; RHS Error 6.086317493390284e-06
Converged. Total iteration 1000: LHS Error: 4.518060615896213e-06; RHS Error 6.086492286613474e-06


In [211]:
ψ_ell = model_res['v0'][:, -1]
ψ_ell_right = model_res_right['v0'][:, -1]
dψ_ell = (ψ_ell_right-ψ_ell)/ell_step
r = -dψ_ell